In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("wyattowalsh/basketball")

print("Path to dataset files:", path)

c:\Users\leona\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 697M/697M [00:23<00:00, 31.1MB/s] 

Extracting files...


Path to dataset files: C:\Users\leona\.cache\kagglehub\datasets\wyattowalsh\basketball\versions\231


In [2]:
# Iniciar uma Sessão Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExemploPySpark") \
    .getOrCreate()

**Comparando métodos do pandas x métodos do PySpark**

Leitura de Dados:

In [3]:
import pandas as pd
df_pandas = pd.read_csv(r"C:\Users\leona\.cache\kagglehub\datasets\wyattowalsh\basketball\versions\231\csv\game.csv")

In [5]:
df_pandas.tail(4)

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type
65694,42022,1610612748,MIA,Miami Heat,42200404,2023-06-09 00:00:00,MIA vs. DEN,L,240,35.0,...,34.0,26.0,11.0,7.0,8.0,18.0,108.0,13,1,Playoffs
65695,42022,1610612743,DEN,Denver Nuggets,42200405,2023-06-12 00:00:00,DEN vs. MIA,W,240,38.0,...,44.0,18.0,9.0,7.0,8.0,21.0,89.0,-5,1,Playoffs
65696,32022,1610616834,LBN,Team LeBron,32200001,2023-02-19 00:00:00,LBN vs. GNS,L,221,79.0,...,46.0,43.0,8.0,1.0,12.0,2.0,184.0,9,1,All-Star
65697,32022,1610616834,LBN,Team LeBron,32200001,2023-02-19 00:00:00,LBN vs. GNS,L,221,79.0,...,46.0,43.0,8.0,1.0,12.0,2.0,184.0,9,1,All Star


In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ExemploLeitura").getOrCreate()

# Leitura de um arquivo CSV
df_pyspark = spark.read.csv(r"C:\Users\leona\.cache\kagglehub\datasets\wyattowalsh\basketball\versions\231\csv\game.csv", header=True, inferSchema=True)

In [7]:
#Nomes das colunas
for i in df_pandas.columns:
  print(i)

season_id
team_id_home
team_abbreviation_home
team_name_home
game_id
game_date
matchup_home
wl_home
min
fgm_home
fga_home
fg_pct_home
fg3m_home
fg3a_home
fg3_pct_home
ftm_home
fta_home
ft_pct_home
oreb_home
dreb_home
reb_home
ast_home
stl_home
blk_home
tov_home
pf_home
pts_home
plus_minus_home
video_available_home
team_id_away
team_abbreviation_away
team_name_away
matchup_away
wl_away
fgm_away
fga_away
fg_pct_away
fg3m_away
fg3a_away
fg3_pct_away
ftm_away
fta_away
ft_pct_away
oreb_away
dreb_away
reb_away
ast_away
stl_away
blk_away
tov_away
pf_away
pts_away
plus_minus_away
video_available_away
season_type


Seleção de Colunas:

In [8]:
# Seleciona colunas específicas - Pandas
df_selecionado_pandas = df_pandas[['team_name_home', 'game_date','matchup_home']]

In [9]:
# Seleciona colunas específicas - PySpark
df_selecionado_spark = df_pyspark.select('team_name_home', 'game_date','matchup_home')

Filtragem de Dados:

In [11]:
# Filtrando o DataFrame para datas após 1 de janeiro de 2023
df_filtrado_pandas = df_pandas[df_pandas['game_date'] > '2023-01-01 00:00:00']

In [12]:
# Filtrando o DataFrame para datas após 1 de janeiro de 2023
df_filtrado_spark = df_pyspark.filter(df_pyspark.game_date > '2023-01-01 00:00:00')

Agregações:

In [13]:
# Calcula a média dos pontos em casa agrupada pelo nome da equipe
df_agregado_pandas = df_pandas.groupby('team_name_home')['pts_home'].mean().reset_index()

In [14]:
from pyspark.sql.functions import avg

# Calcula a média dos pontos em casa (pts_home) agrupada pelo nome da equipe (team_name_home)
df_agregado_spark = df_pyspark.groupBy("team_name_home").agg(avg("pts_home").alias("media_pts_home"))


In [15]:
df_selecionado_spark.show(5)
df_filtrado_spark.show(5)
df_agregado_spark.show(5)

+--------------------+-------------------+------------+
|      team_name_home|          game_date|matchup_home|
+--------------------+-------------------+------------+
|     Toronto Huskies|1946-11-01 00:00:00| HUS vs. NYK|
|   St. Louis Bombers|1946-11-02 00:00:00| BOM vs. PIT|
|Providence Steamr...|1946-11-02 00:00:00| PRO vs. BOS|
|       Chicago Stags|1946-11-02 00:00:00| CHS vs. NYK|
|     Detroit Falcons|1946-11-02 00:00:00| DEF vs. WAS|
+--------------------+-------------------+------------+
only showing top 5 rows

+---------+------------+----------------------+--------------------+--------+-------------------+------------+-------+---+--------+--------+-----------+---------+---------+------------+--------+--------+-----------+---------+---------+--------+--------+--------+--------+--------+-------+--------+---------------+--------------------+------------+----------------------+-----------------+------------+-------+--------+--------+-----------+---------+---------+------------

Junções (Joins):

In [16]:
df1 = pd.read_csv(r"C:\Users\leona\.cache\kagglehub\datasets\wyattowalsh\basketball\versions\231\csv\game.csv")
df2 = pd.read_csv(r"C:\Users\leona\.cache\kagglehub\datasets\wyattowalsh\basketball\versions\231\csv\game_info.csv")
df_juncao = pd.merge(df1, df2, on="game_id", how="left")

In [17]:
# Junção de dois DataFrames PySpark
df1 = spark.read.csv(r"C:\Users\leona\.cache\kagglehub\datasets\wyattowalsh\basketball\versions\231\csv\game.csv", header=True, inferSchema=True)
df2 = spark.read.csv(r"C:\Users\leona\.cache\kagglehub\datasets\wyattowalsh\basketball\versions\231\csv\game_info.csv", header=True, inferSchema=True)
df_juncao = df1.join(df2, on="game_id", how="left")